In [1]:
# from google.colab import drive
# drive.mount('/content/drive')
# import os
# os.chdir('/content/drive/MyDrive/phd/project_af')

In [2]:
# pip install sklearn==0.23.1 

In [3]:
# pip install torch

In [4]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
## Progress bar
from tqdm.notebook import tqdm
from sklearn.preprocessing import OneHotEncoder # need to upgrade to new version

import scipy.io

In [6]:
# load data
data = pd.read_csv("./data/data_9s.csv")

# add a numerical patinet id column
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
pid = le.fit_transform(data.iloc[:,2702])
data = pd.concat([data, pd.DataFrame({'pid':pid})], axis=1)
data.head(5)

,0,1,2,3,4,5,6,7,8,9,...,2695,2696,2697,2698,2699,2700,2701,2702,2703,pid
0,-127,-162,-197,-229,-245,-254,-261,-265,-268,-268,...,-49,-50,-49,-48,-45,N,train,A00001,0,0
1,-41,-35,-28,-19,12,69,151,257,382,522,...,10,10,9,9,8,N,train,A00001,1,0
2,7,7,6,4,3,2,2,3,4,6,...,50,46,42,36,29,N,train,A00001,2,0
3,128,157,189,226,250,257,262,265,268,269,...,0,1,4,7,10,N,test,A00002,0,1
4,13,15,18,21,24,27,31,35,41,48,...,-134,-112,-88,-66,-31,N,test,A00002,1,1


In [7]:
# preprocessing ID column 2702
def preprocess_id(data):

    # Formatting pid
    pid = data.iloc[:,2704]
    pid = torch.tensor(np.array(pid), dtype=torch.int32) # convert pandas DataFrame (x) and numpy array (y) into PyTorch tensors
    
    return pid

In [8]:
# # loop through every 1000 rows per group 
# for g in ["train","valid", "test"]:
#     df = data.loc[data.iloc[:,2701]==g,:]
#     for i in range(df.shape[0]//1000):
#         df_sub = df.iloc[range(i*1000,min((i+1)*1000, df.shape[0])),:]
#         pid = preprocess_id(df_sub)
#         torch.save(pid,"./data/pid_"+g+"_sub"+str(i)+".pth")

In [9]:
def x2spec(x):
    # x is a (batch size, channel=1, signal length) tensor
    x_spec_init = True
    for r in tqdm(range(x.shape[0])):
        from matplotlib.backends.backend_agg import FigureCanvasAgg
        fig, ax = plt.subplots(figsize=(5,3))
        ax.specgram(x[r,0,:], Fs=300)
        plt.axis("off")
        plt.tight_layout()
        plt.close() # no need to show the plot
        canvas = FigureCanvasAgg(fig)
        canvas.draw()
        width, height = fig.get_size_inches() * fig.get_dpi()
        x_3d_spec_r = np.frombuffer(canvas.tostring_rgb(), dtype='uint8').reshape(int(height), int(width), 3)
        x_3d_spec_r = torch.Tensor(x_3d_spec_r).permute(2, 0, 1).unsqueeze(0) # Reshape it to (batch_size=1, channel, height, width)
        x_3d_spec_r = x_3d_spec_r/255.0 # normalize
        if  x_spec_init:
            x_spec = x_3d_spec_r
            # print(x_3d_spec_r.shape)
            x_spec_init = False
        else:
            x_spec = torch.cat((x_spec, x_3d_spec_r), dim=0)
            
            
    # x_spec is a (batch size, channel=3, spectrogram width, spectrogram height) tensor
    return x_spec

In [10]:
# Convert to pytorch tensors
def preprocess(data, x_spec=False):

    # Formatting y
    y = data.iloc[:,[2700]]
    ohe = OneHotEncoder(handle_unknown='ignore', sparse=False).fit(y)
    y = ohe.transform(y)
    y = torch.tensor(y, dtype=torch.float32) # convert pandas DataFrame (x) and numpy array (y) into PyTorch tensors

    # Formatting x in 2 ways
    # way 1: conv1d signal
    x = data.iloc[:,range(9*300)].values
    x = x.astype(float)
    # x = (x - x.mean(axis=1)[:,np.newaxis])/x.std(axis=1)[:,np.newaxis] # standardization by row causing
    # x = (x - np.mean(x))/np.std(x) # nan loss
    for r in range(x.shape[0]): # 0-1 scaling by row
        x[r,:] = (x[r,:] - np.min(x[r,:])) / (np.max(x[r,:]) - np.min(x[r,:])) - 0.5
    assert(np.sum(np.isnan(x))==0)
    x = torch.tensor(x, dtype=torch.float32)
    x = x.view(x.shape[0], 1, x.shape[1])# reshape x to be (batch size, number of channels, signal length) for conv1d

    # way 2: conv2d spectrogram -- slow but work, need to upgrade to spark
    if x_spec:
        x_spec = x2spec(x)
    return x, y, x_spec

In [11]:
from pytictoc import TicToc
t = TicToc() #create instance of class
t.tic() #Start timer
# loop through every 1000 rows per group 
for g in ["test"]: #"train","valid",
    df = data.loc[data.iloc[:,2701]==g,:]
    for i in range(df.shape[0]//1000):
        df_sub = df.iloc[range(i*1000,min((i+1)*1000, df.shape[0])),:]
        x_sub, y_sub, x_spec_sub = preprocess(df_sub, x_spec=True)
#         torch.save(x_sub,"./data/x_"+g+"_sub"+str(i)+".pth")
#         torch.save(x_spec_sub,"./data/x_spec_"+g+"_sub"+str(i)+".pth")
#         torch.save(y_sub,"./data/y_"+g+"_sub"+str(i)+".pth")

t.toc() #Time elapsed since t.tic() # Elapsed time is 165.959995 seconds.

/home/jq2uw/.local/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


<ipython-input-9-485f527edffb>:15: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:206.)
  x_3d_spec_r = torch.Tensor(x_3d_spec_r).permute(2, 0, 1).unsqueeze(0) # Reshape it to (batch_size=1, channel, height, width)


/home/jq2uw/.local/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(



Elapsed time is 165.959995 seconds.


In [1]:

# # x_test, y_test, x_spec_test = preprocess(data.iloc[1:10,:]) # for testing use
# # x, y, x_spec = preprocess(data)
# x_train, y_train, x_spec_train = preprocess(data.loc[data.iloc[:,2701]=="train",:])
# x_valid, y_valid, x_spec_valid = preprocess(data.loc[data.iloc[:,2701]=="valid",:])
# x_test, y_test, x_spec_test = preprocess(data.loc[data.iloc[:,2701]=="test",:])



# # visualize one of the signals in data set
# plt.plot(x_test[5,0,:])
# #plt.axis("off")
# plt.title(str(y_test[5]))
# plt.show();

# # visualize one of the spectrograms in data set
# plt.imshow(x_spec_test[5,:,:,:].permute(1,2,0))
# #plt.axis("off")
# plt.title(str(y_test[5]))
# plt.show
